In [ ]:
import sys
from PyQt6.QtWidgets import (QApplication, QMainWindow, QWidget, QVBoxLayout, QHBoxLayout,
                             QPushButton, QLabel, QTextEdit, QFileDialog, QMessageBox)
from PyQt6.QtGui import QPixmap
from PyQt6.QtCore import Qt
from PIL import Image
from pix2text import Pix2Text
import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from PyQt6.QtGui import QImage, QPixmap 

class Pix2TextApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Reconnaissance de Formules Manuscrites")
        self.setGeometry(100, 100, 800, 600)

        # Initialiser Pix2Text
        self.p2t = Pix2Text.from_config()

        # Widget principal et layout
        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
        self.layout = QVBoxLayout(self.central_widget)
        self.layout.setContentsMargins(20, 20, 20, 20)
        self.layout.setSpacing(15)

        # Titre
        self.title_label = QLabel("Reconnaissance de Formules Manuscrites")
        self.title_label.setStyleSheet("font-size: 16px; font-weight: bold; color: #333;")
        self.title_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.layout.addWidget(self.title_label)

        # Bouton pour choisir l'image
        self.btn_select = QPushButton("Choisir une image")
        self.btn_select.setStyleSheet("""
            QPushButton {
                background-color: #4a90e2;
                color: white;
                padding: 8px;
                border-radius: 5px;
                font-size: 12px;
            }
            QPushButton:hover {
                background-color: #357abd;
            }
        """)
        self.btn_select.clicked.connect(self.load_image)
        self.layout.addWidget(self.btn_select)

        # Zone pour afficher l'image
        self.image_label = QLabel()
        self.image_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.image_label.setStyleSheet("background-color: white; border: 1px solid #d3d3d3; border-radius: 5px;")
        self.image_label.setFixedSize(350, 350)
        self.layout.addWidget(self.image_label)

        # Boîte pour le code LaTeX
        self.latex_output = QTextEdit()
        self.latex_output.setPlaceholderText("Code LaTeX apparaîtra ici...")
        self.latex_output.setStyleSheet("""
            QTextEdit {
                font-family: 'Courier New';
                font-size: 10pt;
                border: 1px solid #d3d3d3;
                border-radius: 5px;
                padding: 5px;
                background-color: #fff;
            }
        """)
        self.latex_output.setFixedHeight(80)
        self.layout.addWidget(self.latex_output)

        # Bouton pour copier le LaTeX
        self.btn_copy = QPushButton("Copier le LaTeX")
        self.btn_copy.setStyleSheet("""
            QPushButton {
                background-color: #4a90e2;
                color: white;
                padding: 8px;
                border-radius: 5px;
                font-size: 12px;
            }
            QPushButton:hover {
                background-color: #357abd;
            }
        """)
        self.btn_copy.clicked.connect(self.copy_latex)
        self.layout.addWidget(self.btn_copy)



        # Ajouter un stretch pour centrer le contenu
        self.layout.addStretch()

        # Style général de la fenêtre
        self.setStyleSheet("background-color: #f5f6f5;")

    def load_image(self):
        file_path, _ = QFileDialog.getOpenFileName(
            self, "Choisir une image", "", "Image files (*.png *.jpg *.jpeg *.bmp)"
        )
        if file_path:
            try:
                # Afficher l'image
                img = Image.open(file_path)
                img = img.resize((350, 350), Image.Resampling.LANCZOS)
                # Convertir l'image PIL en QPixmap
                img = img.convert("RGBA")
                data = img.tobytes("raw", "RGBA")
                qimage = QImage(data, img.size[0], img.size[1], QImage.Format.Format_RGBA8888)
                qpixmap = QPixmap.fromImage(qimage)
                self.image_label.setPixmap(qpixmap)

                # Reconnaître la formule
                outs = self.p2t.recognize_formula(file_path)

                # Afficher le code LaTeX
                self.latex_output.setText(outs)



            except Exception as e:
                QMessageBox.critical(self, "Erreur", f"Erreur : {str(e)}")

    def copy_latex(self):
        latex_text = self.latex_output.toPlainText().strip()
        if latex_text:
            clipboard = QApplication.clipboard()
            clipboard.setText(latex_text)
            QMessageBox.information(self, "Succès", "Code LaTeX copié !")
        else:
            QMessageBox.warning(self, "Avertissement", "Aucun code LaTeX à copier.")

def main():
    app = QApplication(sys.argv)
    window = Pix2TextApp()
    window.show()
    sys.exit(app.exec())

if __name__ == "__main__":
    main()

$$
e^{-n}
$$

$$
\pm\sqrt{b}
$$

$$
z y+2 z y+2 z+2 y
$$

$$
\int_{a}^{x} f ( x ) dx
$$

$$
\sum_{n} {}^{j [ n ]}
$$